In [ ]:
import numpy as np
import copy
import functools
import operator
# import torch

In [ ]:
# mydevice = torch.device('cpu')
# tropicaldt = np.dtype([('E',np.int32),('n',np.int32)])

We define class `Tropical`

In [3]:
class Tropical:
    def __init__(self,e,n):
        self.e = e
        self.n = n
    
    def __add__(self,other):
        if self.e == other.e:
            n1 = self.n+other.n
        elif self.e > other.e:
            n1 = other.n
        elif self.e< other.e:
            n1 = self.n
        return Tropical(min(self.e,other.e),n1)
    
    def __mul__(self,other):
        return Tropical(self.e+other.e,self.n*other.n)
    
    def __str__(self):
        return "Tropical(%d,%d)" %(self.e,self.n)

Try primal way to gert eisum function for objects

In [4]:
def oinsum(eq, *arrays):
    
    """A ``einsum`` implementation for ``numpy`` object arrays.
    """
    lhs, output = eq.split('->')
    inputs = lhs.split(',')

    sizes = {}
    for term, array in zip(inputs, arrays):
        for k, d in zip(term, array.shape):
            sizes[k] = d

    out_size = tuple(sizes[k] for k in output)
    out = np.empty(out_size, dtype=object) # malloc

    inner = [k for k in sizes if k not in output]
    inner_size = [sizes[k] for k in inner]

    for coo_o in np.ndindex(*out_size):

        coord = dict(zip(output, coo_o))

        def gen_inner_sum():
            for coo_i in np.ndindex(*inner_size):
                coord.update(dict(zip(inner, coo_i)))

                locs = [tuple(coord[k] for k in term) for term in inputs]
                elements = (array[loc] for array, loc in zip(arrays, locs))
                yield functools.reduce(operator.mul, elements)

        out[coo_o] = functools.reduce(operator.add, gen_inner_sum())

    return out

In [5]:
if __name__=="__main__":
    B = np.array([[Tropical(1, 1), Tropical(-1, 1)],
                  [Tropical(-1, 1), Tropical(1, 1)]], dtype=object)
    
    G = oinsum('ij,jk,kl,lm,mi->ijklm', B, B, B, B, B)
    Q = oinsum('abcde,ef,fghij->abcdghij',G,B,G)
    # G = object_einsum('ij,jk->ik',B,B)
    # G = oinsum('ij,jk->ik',B,B)

    T = oinsum("abcdefgh,gi,hj,dk->abckefij",Q,B,B,B)

    T2 = oinsum("abcdefgh,hdijgmlk->eabcijklmf",T,T)

    F = oinsum("abcdefghij,gfklhonm,nmlpqcbars->jiosrdekpq",T2,T,T2)

    GB = oinsum("abcde,ai,bj,ck,dl,em->ijklm",G,B,B,B,B,B) # Attach each leg of G with a B matrix
    Emin = oinsum("abcdefghij,abcde,fghij->",F,GB,GB) # get the minimal
    print(Emin)
    pass


Tropical(-66,16000)
